# Specifying epitopes

Polyclonal sera contains antibodies that can target multiple distinct epitopes on a viral protein, but we do not know the exact composition of this sera and which epitopes are targeted. At minimum, `Polyclonal` requires a user to specify the number of epitopes prior to fitting. Alternatively, a user may opt to seed initial guesses for the mutation-escape values of key epitope sites.

Here, we’ll use simulated data to show how the number of epitopes and seeding of key sites can affect the performance of `Polyclonal` models. We'll also use this to provide guidance on how to systematically determine the optimal number of epitopes. 

In [1]:
import itertools
import os
import pickle

import altair as alt
import numpy as np
import pandas as pd
import polyclonal

First, we read in a simulated “noisy” dataset containing 30,000 variants measured at three different sera concentrations. The variants in this library were simulated to contain a Poisson-distributed number of mutations, with an average of three mutations per gene.

In [2]:
noisy_data = (
    pd.read_csv("RBD_variants_escape_noisy.csv", na_filter=None)
    .query('library == "avg3muts"')
    .query("concentration in [0.25, 1, 4]")
    .reset_index(drop=True)
)
noisy_data

,library,aa_substitutions,concentration,prob_escape,IC90
0,avg3muts,,0.25,0.00000,0.1128
1,avg3muts,,0.25,0.01090,0.1128
2,avg3muts,,0.25,0.01458,0.1128
3,avg3muts,,0.25,0.09465,0.1128
4,avg3muts,,0.25,0.03299,0.1128
...,...,...,...,...,...
89995,avg3muts,Y449I L518Y C525R L461I,4.00,0.02197,2.3100
89996,avg3muts,Y449V K529R N394R,4.00,0.04925,0.9473
89997,avg3muts,Y451L N481T F490V,4.00,0.02315,0.9301
89998,avg3muts,Y453R V483G L492V N501P I332P,4.00,0.00000,5.0120


We'll also read in the "true" mutation-escape values, $\beta_{m,e}$, as a reference for later.

In [3]:
true_mut_escape_df = pd.read_csv("RBD_mut_escape_df.csv").replace(
    "class ", "", regex=True
)
true_mut_escape_df

,epitope,mutation,escape
0,1,N331A,0.08226
1,1,N331D,0.77700
2,1,N331E,0.08226
3,1,N331F,0.08226
4,1,N331G,1.70100
...,...,...,...
5791,3,T531R,0.72620
5792,3,T531S,0.72620
5793,3,T531V,0.72480
5794,3,T531W,0.72280


Additionally, we’ll make a directory for storing our fit models as [pickle](https://docs.python.org/3/library/pickle.html#module-pickle) files, so that we can conveniently load them in the future without having to fit again.

In [4]:
os.makedirs("fit_polyclonal_models", exist_ok=True)

## Specifying epitopes without prior knowledge

First, we'll simulate a case in which we have no prior knowledge about which sites confer antibody escape. Thus, we will only supply guesses for the number of epitopes targeted by the polyclonal sera. We can guess that there are probably at least 2 epitopes targeted by polyclonal sera, so we'll start by specifying 2 epitopes and iteratively fit `Polyclonal` models with increasing number of epitopes.  

In [5]:
n_epitopes = [2, 3, 4]

# store fit models for future lookup
fit_mut_escape = {}

for e in n_epitopes:
    # The key for the fit model
    model_string = f"noisy_[0.25, 1, 4]conc_3muts_{e}epitopes_noinit"

    # If the pickled model exists in the fit_polyclonal_models directory, load it.
    if os.path.exists(f"fit_polyclonal_models/{model_string}.pkl") is True:
        model = pickle.load(open(f"fit_polyclonal_models/{model_string}.pkl", "rb"))
        fit_mut_escape.update({model_string: model})
        print(f"Model with {e} epitopes specified was already fit.")
    else:
        # Else, fit a model and save it to the fit_polyclonal_models directory.
        model = polyclonal.Polyclonal(
            data_to_fit=noisy_data,
            n_epitopes=e,
        )
        opt_res = model.fit(logfreq=500)
        fit_mut_escape.update({model_string: model})
        pickle.dump(model, open(f"fit_polyclonal_models/{model_string}.pkl", "wb"))

Model with 2 epitopes specified was already fit.
Model with 3 epitopes specified was already fit.
Model with 4 epitopes specified was already fit.


We can correlate the $\beta_{m,e}$ values for each of the epitopes fit by `Polyclonal` to the $\beta_{m,e}$ values for each of the three "true" epitopes. These $\beta_{m,e}$ values represent the extent to which mutations mediate escape from specific epitopes.

In [6]:
plot_data = pd.DataFrame()
for e in n_epitopes:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{e}epitopes_noinit"]

    # get correlation of mutation-escape values between all pairwise combinations of fit vs. true epitopes
    for true_e, fit_e in list(
        itertools.product(
            true_mut_escape_df["epitope"].unique(),
            model.mut_escape_df["epitope"].unique(),
        )
    ):

        df = true_mut_escape_df.query(f'epitope == "{true_e}"').merge(
            (
                model.mut_escape_df.rename(
                    columns={
                        "escape": "predicted escape",
                        "epitope": "predicted epitope",
                    }
                ).query(f'`predicted epitope` == "{fit_e}"')
            ),
            on="mutation",
            validate="one_to_one",
        )

        corr = df["escape"].corr(df["predicted escape"] ** 2)

        plot_data = pd.concat(
            [
                plot_data,
                pd.DataFrame(
                    data={
                        "Number of fit epitopes": [e],
                        "True epitope class": [true_e],
                        "Fit epitope class": [fit_e],
                        "Correlation (R^2)": [corr],
                    }
                ),
            ]
        ).reset_index(drop=True)

In [7]:
alt.Chart(plot_data).mark_rect(opacity=0.95, stroke="black",).encode(
    x="Fit epitope class:O",
    y="True epitope class:O",
    color=alt.Color("Correlation (R^2):Q", scale=alt.Scale(scheme="viridis")),
    tooltip=[
        "Fit epitope class:O",
        "True epitope class:O",
        alt.Tooltip("Correlation (R^2)", format=".3f"),
    ],
).properties(width=130, height=100,).facet(
    "Number of fit epitopes",
)

alt.FacetChart(...)

We can also visualize the wildtype activity values, $a_{wt,e}$.

In [8]:
for e in n_epitopes:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{e}epitopes_noinit"]
    print(f"Number of fit epitopes: {e}")
    display(model.activity_wt_barplot())

Number of fit epitopes: 2


alt.Chart(...)

Number of fit epitopes: 3


alt.Chart(...)

Number of fit epitopes: 4


alt.Chart(...)

In the simulation ground truth, there are three epitopes (class 1, 2, and 3). The order of immunodominance from highest to lowest is class 2 > class 3 > class 1.

- The model that specified two epitopes (left heatmap) fit mutation-escape values that strongly correlate with that of the true class 2 and 3 epitopes. This is expected, as the model prioritizes epitopes by their immunodominance and ignores the most subdominant class 1 epitope.

- The model that specified three epitopes (center heatmap) fit mutation-escape values that strongly correlate with that of the true class 1, 2, and 3 epitopes. This is also expected, suggesting that the class 1 epitope, though subdominant, is not redundant. 

- The model that specified four epitopes (right heatmap) fit three epitopes with mutation-escape values that strongly correlate with that of the true class 1, 2, and 3 epitopes. However, the fourth fit epitope did not correlate with any of the true epitopes. This "redundant" epitope also had a strongly negative wildtype activity value, suggesting that antibodies targeting that "epitope" don't make an appreciable contribution to the observed data.

Additionally, we can look at the correlation between “true” and predicted IC90’s for each of the fit models. To do this, we’ll predict the IC90’s of variants in a separate library with a different (higher) mutation rate. We therefore read in the “exact” simulated data from a library containing variants with an average of four mutations per gene.

In [9]:
exact_data = (
    pd.read_csv("RBD_variants_escape_exact.csv", na_filter=None)
    .query('library == "avg4muts"')
    .query("concentration in [1]")
    .reset_index(drop=True)
)

We’ll make the comparison on a log scale, and clip IC90s at values >50 as that is likely to be way outside the dynamic range given the concentrations used.

In [10]:
ic90_corrs = pd.DataFrame()

max_ic90 = 50
for e in n_epitopes:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{e}epitopes_noinit"]

    ic90s = (
        exact_data[["aa_substitutions", "IC90"]]
        .assign(IC90=lambda x: x["IC90"].clip(upper=max_ic90))
        .drop_duplicates()
    )
    ic90s = model.filter_variants_by_seen_muts(ic90s)
    ic90s = model.icXX(ic90s, x=0.9, col="predicted_IC90", max_c=max_ic90)

    ic90s = ic90s.assign(
        log_IC90=lambda x: np.log10(x["IC90"]),
        predicted_log_IC90=lambda x: np.log10(x["predicted_IC90"]),
    )

    corr = ic90s["log_IC90"].corr(ic90s["predicted_log_IC90"]) ** 2

    ic90_corrs = pd.concat(
        [
            ic90_corrs,
            pd.DataFrame(
                {
                    "Correlation (R^2)": corr,
                    "Number of fit epitopes": [e],
                }
            ),
        ]
    )

In [11]:
# NBVAL_IGNORE_OUTPUT
alt.Chart(ic90_corrs).mark_circle(size=125).encode(
    x="Number of fit epitopes:O",
    y="Correlation (R^2):Q",
    tooltip=["Number of fit epitopes", alt.Tooltip("Correlation (R^2)", format=".3f")],
).properties(width=200, height=150, title="predicted vs. true IC90")

alt.Chart(...)

Although the IC90 prediction appears excellent regardless of the number of fit epitopes, adding the 3rd epitope does lead to a small improvement in the prediction.

## Specifying epitopes with prior knowledge

Next, we'll explore the consequences of cases where we think we have prior knowledge of some sites that confer antibody escape, but do not know the exact number of epitopes that are targeted.

We can start by "seeding" initial guesses for 2 epitopes and iteratively fitting `Polyclonal` models with more initial guesses. We know from prior knowledge that site 484 and site 444 are key epitope sites, so we seed them with large mutation-escape values.

In [12]:
# The key for the fit model
model_string = "noisy_[0.25, 1, 4]conc_3muts_2epitopes_seed484_444"

# If the pickled model exists in the fit_polyclonal_models directory, load it.
if os.path.exists(f"fit_polyclonal_models/{model_string}.pkl") is True:
    model = pickle.load(open(f"fit_polyclonal_models/{model_string}.pkl", "rb"))
    fit_mut_escape.update({model_string: model})
    print(f"Model with 2 epitopes seeded at 484 and 444 was already fit.")
else:
    # Else, fit a model and save it to the fit_polyclonal_models directory.
    model = polyclonal.Polyclonal(
        data_to_fit=noisy_data,
        activity_wt_df=pd.DataFrame.from_records(
            [
                ("1", 2.0),
                ("2", 1.0),
            ],
            columns=["epitope", "activity"],
        ),
        site_escape_df=pd.DataFrame.from_records(
            [
                ("1", 484, 10.0),
                ("2", 444, 10.0),
            ],
            columns=["epitope", "site", "escape"],
        ),
        data_mut_escape_overlap="fill_to_data",
    )
    opt_res = model.fit(logfreq=500)
    fit_mut_escape.update({model_string: model})
    pickle.dump(model, open(f"fit_polyclonal_models/{model_string}.pkl", "wb"))

Model with 2 epitopes seeded at 484 and 444 was already fit.


Now, we'll fit a `Polyclonal` model with 3 epitopes by seeding an additional key site, 417, which is often considered to be in the most subdominant class 1 epitope.

In [13]:
# The key for the fit model
model_string = "noisy_[0.25, 1, 4]conc_3muts_3epitopes_seed417_484_444"

# If the pickled model exists in the fit_polyclonal_models directory, load it.
if os.path.exists(f"fit_polyclonal_models/{model_string}.pkl") is True:
    model = pickle.load(open(f"fit_polyclonal_models/{model_string}.pkl", "rb"))
    fit_mut_escape.update({model_string: model})
    print(f"Model with 3 epitopes seeded at 417, 484, and 444 was already fit.")
else:
    # Else, fit a model and save it to the fit_polyclonal_models directory.
    model = polyclonal.Polyclonal(
        data_to_fit=noisy_data,
        activity_wt_df=pd.DataFrame.from_records(
            [
                ("1", 1.0),
                ("2", 3.0),
                ("3", 2.0),
            ],
            columns=["epitope", "activity"],
        ),
        site_escape_df=pd.DataFrame.from_records(
            [
                ("1", 417, 10.0),
                ("2", 484, 10.0),
                ("3", 444, 10.0),
            ],
            columns=["epitope", "site", "escape"],
        ),
        data_mut_escape_overlap="fill_to_data",
    )
    opt_res = model.fit(logfreq=500)
    fit_mut_escape.update({model_string: model})
    pickle.dump(model, open(f"fit_polyclonal_models/{model_string}.pkl", "wb"))

Model with 3 epitopes seeded at 417, 484, and 444 was already fit.


Next, we'll fit a `Polyclonal` model with 4 epitopes and try seeding an additional key site, 386, that is **not** actually in an epitope that is targeted by polyclonal antibodies.

In [14]:
# The key for the fit model
model_string = "noisy_[0.25, 1, 4]conc_3muts_4epitopes_seed417_484_444_386"

# If the pickled model exists in the fit_polyclonal_models directory, load it.
if os.path.exists(f"fit_polyclonal_models/{model_string}.pkl") is True:
    model = pickle.load(open(f"fit_polyclonal_models/{model_string}.pkl", "rb"))
    fit_mut_escape.update({model_string: model})
    print(f"Model with 4 epitopes seeded at 417, 484, 444, and 386 was already fit.")
else:
    # Else, fit a model and save it to the fit_polyclonal_models directory.
    model = polyclonal.Polyclonal(
        data_to_fit=noisy_data,
        activity_wt_df=pd.DataFrame.from_records(
            [
                ("1", 2.0),
                ("2", 4.0),
                ("3", 3.0),
                ("4", 1.0),
            ],
            columns=["epitope", "activity"],
        ),
        site_escape_df=pd.DataFrame.from_records(
            [
                ("1", 417, 10.0),
                ("2", 484, 10.0),
                ("3", 444, 10.0),
                ("4", 386, 10.0),
            ],
            columns=["epitope", "site", "escape"],
        ),
        data_mut_escape_overlap="fill_to_data",
    )
    opt_res = model.fit(logfreq=500)
    fit_mut_escape.update({model_string: model})
    pickle.dump(model, open(f"fit_polyclonal_models/{model_string}.pkl", "wb"))

Model with 4 epitopes seeded at 417, 484, 444, and 386 was already fit.


Lastly, we'll fit a `Polyclonal` model with 4 epitopes and try seeding an additional key site that is already located in an epitope that we've specified. Specifically, we'll seed it with site 460, which is in the class 1 epitope along with site 417.

In [15]:
# The key for the fit model
model_string = "noisy_[0.25, 1, 4]conc_3muts_4epitopes_seed417_484_444_460"

# If the pickled model exists in the fit_polyclonal_models directory, load it.
if os.path.exists(f"fit_polyclonal_models/{model_string}.pkl") is True:
    model = pickle.load(open(f"fit_polyclonal_models/{model_string}.pkl", "rb"))
    fit_mut_escape.update({model_string: model})
    print(f"Model with 4 epitopes seeded at 417, 484, 444, and 460 was already fit.")
else:
    # Else, fit a model and save it to the fit_polyclonal_models directory.
    model = polyclonal.Polyclonal(
        data_to_fit=noisy_data,
        activity_wt_df=pd.DataFrame.from_records(
            [
                ("1", 2.0),
                ("2", 4.0),
                ("3", 3.0),
                ("4", 1.0),
            ],
            columns=["epitope", "activity"],
        ),
        site_escape_df=pd.DataFrame.from_records(
            [
                ("1", 417, 10.0),
                ("2", 484, 10.0),
                ("3", 444, 10.0),
                ("4", 460, 10.0),
            ],
            columns=["epitope", "site", "escape"],
        ),
        data_mut_escape_overlap="fill_to_data",
    )
    opt_res = model.fit(logfreq=500)
    fit_mut_escape.update({model_string: model})
    pickle.dump(model, open(f"fit_polyclonal_models/{model_string}.pkl", "wb"))

Model with 4 epitopes seeded at 417, 484, 444, and 460 was already fit.


Like before, we can correlate the $\beta_{m,e}$ values for each of the epitopes fit by these `Polyclonal` models to the $\beta_{m,e}$ values for each of the three "true" epitopes.

In [16]:
plot_data = pd.DataFrame()
for st in [
    "2epitopes_seed484_444",
    "3epitopes_seed417_484_444",
    "4epitopes_seed417_484_444_386",
    "4epitopes_seed417_484_444_460",
]:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{st}"]
    for true_e, fit_e in list(
        itertools.product(
            true_mut_escape_df["epitope"].unique(),
            model.mut_escape_df["epitope"].unique(),
        )
    ):

        df = true_mut_escape_df.query(f'epitope == "{true_e}"').merge(
            (
                model.mut_escape_df.rename(
                    columns={
                        "escape": "predicted escape",
                        "epitope": "predicted epitope",
                    }
                ).query(f'`predicted epitope` == "{fit_e}"')
            ),
            on="mutation",
            validate="one_to_one",
        )

        corr = df["escape"].corr(df["predicted escape"] ** 2)

        plot_data = pd.concat(
            [
                plot_data,
                pd.DataFrame(
                    data={
                        "Epitope seeding": [st],
                        "True epitope class": [true_e],
                        "Fit epitope class": [fit_e],
                        "Correlation (R^2)": [corr],
                    }
                ),
            ]
        ).reset_index(drop=True)

In [17]:
alt.Chart(plot_data).mark_rect(opacity=0.95, stroke="black",).encode(
    x="Fit epitope class:O",
    y="True epitope class:O",
    color=alt.Color("Correlation (R^2):Q", scale=alt.Scale(scheme="viridis")),
    tooltip=[
        "Fit epitope class:O",
        "True epitope class:O",
        alt.Tooltip("Correlation (R^2)", format=".3f"),
    ],
).properties(width=130, height=100,).facet(
    "Epitope seeding",
    columns=2,
)

alt.FacetChart(...)

We can also visualize the wildtype activity values, $a_{wt,e}$.

In [18]:
for st in [
    "2epitopes_seed484_444",
    "3epitopes_seed417_484_444",
    "4epitopes_seed417_484_444_386",
    "4epitopes_seed417_484_444_460",
]:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{st}"]
    print(f"Epitope seeding: {st}")
    display(model.activity_wt_barplot())

Epitope seeding: 2epitopes_seed484_444


alt.Chart(...)

Epitope seeding: 3epitopes_seed417_484_444


alt.Chart(...)

Epitope seeding: 4epitopes_seed417_484_444_386


alt.Chart(...)

Epitope seeding: 4epitopes_seed417_484_444_460


alt.Chart(...)

These results look similar to the ones obtained without seeding sites above. But importantly, we can see that the models are not negatively impacted by seeding sites that are not within a targeted epitope or within the same epitope as an already seeded  site. Although not seen on this simulated dataset, keep in mind that seeding sites could be helpful and lead to better results if the library mutation rate is low.

Additionally, we can look at the correlation between “true” and predicted IC90’s for each of these fit models.

In [19]:
ic90_corrs = pd.DataFrame()

max_ic90 = 50
for st in [
    "2epitopes_seed484_444",
    "3epitopes_seed417_484_444",
    "4epitopes_seed417_484_444_386",
    "4epitopes_seed417_484_444_460",
]:
    model = fit_mut_escape[f"noisy_[0.25, 1, 4]conc_3muts_{st}"]

    ic90s = (
        exact_data[["aa_substitutions", "IC90"]]
        .assign(IC90=lambda x: x["IC90"].clip(upper=max_ic90))
        .drop_duplicates()
    )
    ic90s = model.filter_variants_by_seen_muts(ic90s)
    ic90s = model.icXX(ic90s, x=0.9, col="predicted_IC90", max_c=max_ic90)

    ic90s = ic90s.assign(
        log_IC90=lambda x: np.log10(x["IC90"]),
        predicted_log_IC90=lambda x: np.log10(x["predicted_IC90"]),
    )

    corr = ic90s["log_IC90"].corr(ic90s["predicted_log_IC90"]) ** 2

    ic90_corrs = pd.concat(
        [
            ic90_corrs,
            pd.DataFrame(
                {
                    "Correlation (R^2)": corr,
                    "Epitope seeding": [st],
                }
            ),
        ]
    )

In [20]:
# NBVAL_IGNORE_OUTPUT
alt.Chart(ic90_corrs).mark_circle(size=125).encode(
    x="Epitope seeding:O",
    y="Correlation (R^2):Q",
    tooltip=["Epitope seeding", alt.Tooltip("Correlation (R^2)", format=".3f")],
).properties(width=200, height=150, title="predicted vs. true IC90")

alt.Chart(...)

## Summary

These simulation experiments provide a general guideline for specifying epitopes. When fitting `Polyclonal` models, one can start with 2 epitopes and iteratively fit models with increasing number of epitopes. At some point, the newly seeded $N$-th epitope will become redundant, as evidenced by a strongly negative wildtype activity value, $a_{wt,e}$. This is indication to the user that the previous fit model, containing $N - 1$ epitopes, is the one that best captures the data and polyclonal mix.